<a href="https://colab.research.google.com/github/appliedcode/mthree-c422/blob/main/Exercises/day-7/ETL-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL Pipeline in Google Colab Using Only Local Resources
**Objective:**
Learn how to perform a full Extract-Transform-Load (ETL) data pipeline using only Python, pandas, and SQLite—all within Google Colab, with no cloud databases or external APIs required.

## 1. Prerequisites
- A Google account to access Colab.

- Basic Python and pandas knowledge.

- No external cloud or database accounts required.

## 2. Scenario
- You’re provided with a local .csv file (customers.csv) mimicking customer biological measurements. \
- You’ll ingest, clean, enrich, and store this data in a local SQLite database—all inside your Colab notebook.

In [19]:
# A. Extract
# 1. Download or Create a Sample Data File
# Use any small public .csv (for demo, we’ll use a height/weight dataset):
url = "https://people.sc.fsu.edu/~jburkardt/data/csv/hw_200.csv"
!wget -qO customers.csv $url


In [20]:
# 2. Load Into pandas DataFrame
import pandas as pd

df_customers = pd.read_csv('customers.csv')
print(df_customers.head())
print(df_customers.columns)


   Index   Height(Inches)"   "Weight(Pounds)"
0      1             65.78             112.99
1      2             71.52             136.49
2      3             69.40             153.03
3      4             68.22             142.34
4      5             67.79             144.30
Index(['Index', ' Height(Inches)"', ' "Weight(Pounds)"'], dtype='object')


In [21]:
# B. Transform
# 1. Clean Data
# Remove duplicates

# Rename messy columns for cleaner analysis

# Remove duplicates based on 'Index' column
df_customers.drop_duplicates('Index', inplace=True)

# Clean up column names
df_customers.columns = [col.strip().replace('"', '').replace("'", "") for col in df_customers.columns]

# View cleaned DataFrame
print(df_customers.head())


   Index  Height(Inches)  Weight(Pounds)
0      1           65.78          112.99
1      2           71.52          136.49
2      3           69.40          153.03
3      4           68.22          142.34
4      5           67.79          144.30


In [25]:
# 2. Add Enriched & Derived Columns
# Compute a "BMI" (Body Mass Index) column

# Add a BMI column, Height in inches, Weight in pounds
df_customers['BMI'] = (df_customers['Weight(Pounds)'] / (df_customers['Height(Inches)'] * 0.0254) ** 2)
print(df_customers.head())


   Index  Height(Inches)  Weight(Pounds)        BMI
0      1           65.78          112.99  40.474812
1      2           71.52          136.49  41.359796
2      3           69.40          153.03  49.248182
3      4           68.22          142.34  47.406304
4      5           67.79          144.30  48.670704


In [26]:
# C. Load
# 1. Store the Result in a Local SQLite Database
import sqlite3

conn = sqlite3.connect('customers_etl_lab.db')
df_customers.to_sql('customers_etl', conn, if_exists='replace', index=False)


200

In [27]:
# 2. Run SQL Queries on the Transformed Data
# Example: Get customers with BMI > 25 (overweight threshold)
query = "SELECT * FROM customers_etl WHERE BMI > 25;"
df_overweight = pd.read_sql(query, conn)
print(df_overweight.head())


   Index  Height(Inches)  Weight(Pounds)        BMI
0      1           65.78          112.99  40.474812
1      2           71.52          136.49  41.359796
2      3           69.40          153.03  49.248182
3      4           68.22          142.34  47.406304
4      5           67.79          144.30  48.670704
